In [1]:
import os, itertools, subprocess
import numpy as np
import pandas as pd

from mpire import WorkerPool

RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
CROSS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Cross_Analysis_Results/rawCross/"
CROSS_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Cross_Analysis_Results/CrossRes/"

In [2]:
def getINDCrossRes(p1_name):
    
    output_file = pd.DataFrame(columns=["P1", "P2", "P1_ind_score", "P2_score", "Optimal_score", 
                                        "Delta_score", "Same", "Keep", "Replaced_by_P2"])
    
    haplotype_score_info = pd.read_table(os.path.join(RES_PATH, "Haplotype_Score_Info.tsv"), index_col="Haplotype_ID")
    haplotype_score_info = haplotype_score_info['Hap_Score.by_Total_pop'].copy()
    
    samp_score_info = pd.read_table(os.path.join(RES_PATH, "Samples_Score_Info.tsv"))
    samp_list = samp_score_info["Sample"].to_list()
    samp_list.remove(p1_name)
    
    for item in samp_list:
        p1_score = samp_score_info.loc[samp_score_info["Sample"]==p1_name]["Hap_Score.by_Total_pop"].values[0]
        p2_score = samp_score_info.loc[samp_score_info["Sample"]==item]["Hap_Score.by_Total_pop"].values[0]
    
        file_path = os.path.join(CROSS_PATH, "{}/{}_{}.Cross.Info".format(p1_name, p1_name, item))
        raw_tmp_data = pd.read_table(file_path)

        opt_hap_list = [x for x in raw_tmp_data["Optimal_Haplotype"].to_list() if x != "-"]
        opt_hap_score = haplotype_score_info.loc[opt_hap_list].sum()/854.768527747741*100

        delta_ind_score = opt_hap_score - p1_score

        same_num = raw_tmp_data[raw_tmp_data["P1_Breeding_Operation"]=="Same"].shape[0]
        keep_num = raw_tmp_data[raw_tmp_data["P1_Breeding_Operation"]=="Keep"].shape[0]
        replace_num = raw_tmp_data[raw_tmp_data["P1_Breeding_Operation"]=="Replaced_by_P2"].shape[0]

        output_file.loc[len(output_file)] = [p1_name, item, p1_score, p2_score, opt_hap_score, 
                                             delta_ind_score, same_num, keep_num, replace_num]

    output_file = output_file.sort_values(by=["Optimal_score"], ascending=[False])
    output_file.to_csv(os.path.join(CROSS_RES_PATH, "{}.cross.res".format(p1_name)), index=0, sep="\t")
    return
    

if __name__ == "__main__":
    p1_sample_list = os.listdir(CROSS_PATH)

    with WorkerPool(n_jobs=24) as pool:
        pool.map(getINDCrossRes, p1_sample_list, progress_bar=True)
    
    print("ALL DONE!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [03:18<00:00,  2.15it/s]


ALL DONE!
